In [1]:
import wget
import os.path
import json


import numpy as np
from numpy.linalg import det, inv
from numpy import array
import random

import ase.db
import pandas as pd

import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'wget'

In [2]:
!pip install wget


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=290da8539b3d7cdf4f91b9c6e699653c6224a47efb8ea441af87e984779d4a57
  Stored in directory: c:\users\tomut\appdata\local\pip\cache\wheels\04\5f\3e\46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


In [2]:
from anneling_super_cell import build_annealing

In [3]:
DATA_PATH = "/Users/voicutu/Documents/GitHub/vdw_evolve/sandbox/database/"
database_name1 = "c2db-20211702.db"
database_name2 = "c2db-20211702.db"

In [4]:
def get_data_as_pd(path,options,props):
    db = ase.db.connect(path)
    rows = db.select(options)
    data = [ [ x.get(p) for p in props ] for x in rows]
    raw_df = pd.DataFrame( data, columns = props);
    return raw_df 

In [5]:
path = DATA_PATH+database_name1

options = 'is_magnetic=True, thermodynamic_stability_level=3'
props = ["formula","spgnum", "spacegroup","uid","asr_id"] 

raw_df_1 = get_data_as_pd(path,options,props)
raw_df_1 = raw_df_1[ (raw_df_1["spacegroup"]!='P1') & (raw_df_1["spacegroup"]!='Pc')& (raw_df_1["spacegroup"]!='P-1') ];


#Print
print("The number of elements is:",len(list(raw_df_1)) )
print( raw_df_1[["formula","uid"]] )

The number of elements is: 5
       formula                     uid
0       Mn2Se2     Mn2Se2-a1d595c25fc5
1        Ru2F8      Ru2F8-5b1d25d726e0
2         V2F8       V2F8-6d78fbe605b3
3     Bi2MnTe4   MnBi2Te4-a8bb2517b2b7
4      CrGa2S4    CrGa2S4-e6aae0f0d940
..         ...                     ...
394     Cr2Cl6     Cr2Cl6-9f9e75488d50
395     Pt2Cl6     Pt2Cl6-20b975a3a1e8
396    V2Br2N2    Br2N2V2-186b2da4920f
397  Mn2Cl2Se2  Cl2Mn2Se2-6bdadd86e6c4
400       VSSe       SSeV-0f3ba3ea815f

[337 rows x 2 columns]


In [6]:
path = DATA_PATH+database_name2

options = 'thermodynamic_stability_level=3'
props = ["formula","spgnum", "spacegroup","gap","uid"] 

raw_df_2 = get_data_as_pd(path,options,props)
raw_df_2 = raw_df_2[ (raw_df_2["gap"]<=10e-3)& (raw_df_2["spacegroup"]!='P1') & (raw_df_2["spacegroup"]!='Pc')& (raw_df_2["spacegroup"]!='P-1') ];


#Print
print("The number of elements is:",len(list(raw_df_2)) )
print( raw_df_2[["formula","uid","gap"]] )

The number of elements is: 5
         formula                      uid  gap
2         Mn2Se2      Mn2Se2-a1d595c25fc5  0.0
6         Zr2Cl2      Cl2Zr2-b8382c030a92  0.0
11       CrGa2S4     CrGa2S4-e6aae0f0d940  0.0
18    Nb6Si2Te12  Si2Nb6Te12-22d3ae87208a  0.0
19      Nb2PdSe6    PdNb2Se6-2c7db27c6af6  0.0
...          ...                      ...  ...
1962     Mo2I2N2     I2Mo2N2-86ca1139b89c  0.0
1963    Pd2I2Se2    I2Pd2Se2-aaba1a6be8bd  0.0
1965    Na2Ni2P2    Na2Ni2P2-0a96a1dd75a7  0.0
1967       ZrSTe       STeZr-3f7c4bfed2d9  0.0
1971     Co2P2S6     Co2P2S6-a75c2d7a9813  0.0

[765 rows x 3 columns]


In [7]:
# get jsons 
def extract_json(uid_list,save_path ="JSONcolection"):
    for uid in uid_list:
        dataurl = 'https://cmrdb.fysik.dtu.dk/c2db/row/'+uid+'/all_data';
        file = save_path+"/"+uid+"_data.json"
        if os.path.isfile(file):
            print("file: ",file, "found")
        else:
            print(wget.download(dataurl, out="./"+save_path))

def extract_structure(uid_list ,save_path="XYZcolection"):
    for uid in uid_list:
        
        #https://cmrdb.fysik.dtu.dk/c2db/row/N2O2V3-358facb64a22/data/structure.json
        dataurl = 'https://cmrdb.fysik.dtu.dk/c2db/row/'+uid+'/data/structure.json';
        print(dataurl)
        file = save_path+"/"+uid+".json"
        dw_path = save_path+"/"+"structure.json"
        if os.path.isfile(file):
            print("file: ",file, "found")
        else:
            print(wget.download(dataurl, out="./"+save_path))
            os.rename(dw_path, file)

In [8]:
print("extract_data1:")
print(raw_df_1["uid"][:1])
extract_structure(raw_df_1["uid"][:10])
print("extract_data2:")
extract_structure(raw_df_1["uid"][:10])

extract_data1:
0    Mn2Se2-a1d595c25fc5
Name: uid, dtype: object
https://cmrdb.fysik.dtu.dk/c2db/row/Mn2Se2-a1d595c25fc5/data/structure.json
file:  XYZcolection/Mn2Se2-a1d595c25fc5.json found
https://cmrdb.fysik.dtu.dk/c2db/row/Ru2F8-5b1d25d726e0/data/structure.json
file:  XYZcolection/Ru2F8-5b1d25d726e0.json found
https://cmrdb.fysik.dtu.dk/c2db/row/V2F8-6d78fbe605b3/data/structure.json
file:  XYZcolection/V2F8-6d78fbe605b3.json found
https://cmrdb.fysik.dtu.dk/c2db/row/MnBi2Te4-a8bb2517b2b7/data/structure.json
file:  XYZcolection/MnBi2Te4-a8bb2517b2b7.json found
https://cmrdb.fysik.dtu.dk/c2db/row/CrGa2S4-e6aae0f0d940/data/structure.json
file:  XYZcolection/CrGa2S4-e6aae0f0d940.json found
https://cmrdb.fysik.dtu.dk/c2db/row/F2N3V4-3f5b11d43a1b/data/structure.json
100% [............................................................] 1794 / 1794./XYZcolection/structure.json
https://cmrdb.fysik.dtu.dk/c2db/row/N3Sc4-daf440ed0646/data/structure.json
100% [..................................

In [9]:
# Example of good practice in data hadeling 
# uid= "C2-a6735a4a3797"
# with open('XYZcolection/structure.json', 'r') as file:
#     json_data= json.load(file)
# mytype= "__ndarray__";
# datastruc=json_data['1']["cell"]["array"];
# if mytype in datastruc:
#     shape= datastruc[mytype][0]
#     dtype= datastruc[mytype][1]
#     lattice = array(datastruc[mytype][2],dtype=dtype).reshape(shape)


In [10]:
#f = "/Users/voicutu/Documents/GitHub/vdw_evolve/sandbox/XYZcolection/c2db-822.xyz"

def get_lattice_from_structure(f):
    with open(f, 'r') as file:
        json_data= json.load(file)
    mytype= "__ndarray__";
    datastruc=json_data['1']["cell"]["array"];
    if mytype in datastruc:
        shape= datastruc[mytype][0]
        dtype= datastruc[mytype][1]
        lattice = array(datastruc[mytype][2],dtype=dtype).reshape(shape)
    return lattice 

In [35]:
#
up = 99999999
bond =20
model_par = {
    
    'initialTemp': 4, 
    'finalTemp': 0.0002,

    'beta': 10,
    'bounds': [[-2*bond,2*bond] for _ in range(4)],

    'nr_neighbours': 1,
    'step_size': 4,
    'gaussian_sd':3,
    
    'known_min': -up,
    "start_point":[1,5,3,2]
}



In [36]:
uids=["C2-a6735a4a3797","WS2-64090c9845f8"]
extract_structure(uids)

https://cmrdb.fysik.dtu.dk/c2db/row/C2-a6735a4a3797/data/structure.json
file:  XYZcolection/C2-a6735a4a3797.json found
https://cmrdb.fysik.dtu.dk/c2db/row/WS2-64090c9845f8/data/structure.json
file:  XYZcolection/WS2-64090c9845f8.json found


In [40]:
# xyz path 

structure_path = "/Users/voicutu/Documents/GitHub/vdw_evolve/sandbox/XYZcolection"
uid_1 = "WS2-64090c9845f8"#"VCl2-1ccb0b28392d"
uid_2 = "C2-a6735a4a3797"#"Mn2Se2-a1d595c25fc5"


path1=structure_path+"/"+uid_1+".json"
path2=structure_path+"/"+uid_2+".json"

# lattices
latice1 = get_lattice_from_structure(path1)
latice2 = get_lattice_from_structure(path2)


cel1 = np.array([[latice1[0][0], latice1[1][0]],
        [latice1[0][1], latice1[1][1]]])

cel2 = np.array([[latice2[0][0], latice2[1][0]],
        [latice2[0][1], latice2[1][1]]])

tA, tB, strain = build_annealing(cel1,cel2, 7, model_par)
new = np.dot(tA, cel1)

print("\ncel1",cel1)
print("cel2",cel2)
print("\n _____\n tA:\n",tA)
print("det:",det(tA))
print("\n _____\n tB:\n",tB)
print("det:",det(tB))
super_cell= np.dot(tA,cel1)
print("\n _____\n Super cell:\n",super_cell)
print("det:",det(super_cell))
print("\n _____\n Strain:\n",strain)
suuper_cell= np.dot(tA,cel1)



# print("\n zero_mat:\n",zero_mat)

temp:4|epoch:0|change:-14905556284385.61|value:8460540088307.247
temp:4|epoch:5|change:-4524736241743.878|value:3935803846563.369
temp:0.0975609756097561|epoch:0|change:28197106854528.17|value:3935803846563.369
temp:0.0975609756097561|epoch:5|change:14947787748477.928|value:854508279609.265
temp:0.04938271604938272|epoch:0|change:24889482313046.555|value:854508279609.265
temp:0.04938271604938272|epoch:5|change:-452901216904.4046|value:401607062704.8604
temp:0.03305785123966942|epoch:0|change:12766460812057.469|value:401607062704.8604
temp:0.03305785123966942|epoch:5|change:1238143422212.2607|value:401607062704.8604
temp:0.02484472049689441|epoch:0|change:11130074324211.922|value:401607062704.8604
temp:0.02484472049689441|epoch:5|change:7842645303321.509|value:401607062704.8604
temp:0.01990049751243781|epoch:0|change:2781591937764.264|value:401607062704.8604
temp:0.01990049751243781|epoch:5|change:13619285861083.594|value:401607062704.8604
temp:0.016597510373443983|epoch:0|change:370901

temp:0.0019990004997501253|epoch:0|change:24398153866178.29|value:31538672825.294876
temp:0.0019990004997501253|epoch:5|change:4501561271509.616|value:31538672825.294876
temp:0.0019598236158745717|epoch:0|change:26917331158966.586|value:31538672825.294876
temp:0.0019598236158745717|epoch:5|change:20751883676058.906|value:31538672825.294876
temp:0.0019221528111484868|epoch:0|change:33167029238211.605|value:31538672825.294876
temp:0.0019221528111484868|epoch:5|change:9114968805426.754|value:31538672825.294876
temp:0.0018859028760018863|epoch:0|change:8569149793829.878|value:31538672825.294876
temp:0.0018859028760018863|epoch:5|change:3413726842506.5386|value:31538672825.294876
temp:0.0018509949097639986|epoch:0|change:13905965136205.963|value:31538672825.294876
temp:0.0018509949097639986|epoch:5|change:18412856104827.977|value:31538672825.294876
temp:0.0018173557473875517|epoch:0|change:9709241368941.94|value:31538672825.294876
temp:0.0018173557473875517|epoch:5|change:36874763100843.27|

temp:0.0009431737797689218|epoch:5|change:33223506750527.01|value:31538672825.294876
temp:0.0009343611305769675|epoch:0|change:17599147519077.99|value:31538672825.294876
temp:0.0009343611305769675|epoch:5|change:10002526178772.457|value:31538672825.294876
temp:0.0009257116408238829|epoch:0|change:6320327530198.293|value:31538672825.294876
temp:0.0009257116408238829|epoch:5|change:10941082494100.941|value:31538672825.294876
temp:0.0009172208209126342|epoch:0|change:29677564569198.902|value:31538672825.294876
temp:0.0009172208209126342|epoch:5|change:6760187226159.823|value:31538672825.294876
temp:0.000908884344467166|epoch:0|change:15667762851920.322|value:31538672825.294876
temp:0.000908884344467166|epoch:5|change:12215876881853.814|value:31538672825.294876
temp:0.0009006980409817604|epoch:0|change:9349743332872.3|value:31538672825.294876
temp:0.0009006980409817604|epoch:5|change:3641470397607.0024|value:31538672825.294876
temp:0.0008926578888640924|epoch:0|change:12648267339611.309|va

temp:0.0005746300818847864|epoch:0|change:18317333525102.457|value:31538672825.294876
temp:0.0005746300818847864|epoch:5|change:6405268326536.932|value:31538672825.294876
temp:0.0005713469504356518|epoch:0|change:11797384564626.006|value:31538672825.294876
temp:0.0005713469504356518|epoch:5|change:158529920428595.38|value:31538672825.294876
temp:0.0005681011219997157|epoch:0|change:5168428799316.271|value:31538672825.294876
temp:0.0005681011219997157|epoch:5|change:35262518049707.766|value:31538672825.294876
temp:0.0005648919644118059|epoch:0|change:21139432539955.797|value:31538672825.294876
temp:0.0005648919644118059|epoch:5|change:8668143843365.289|value:31538672825.294876
temp:0.0005617188597107145|epoch:0|change:13801519544910.611|value:31538672825.294876
temp:0.0005617188597107145|epoch:5|change:13718814287164.805|value:31538672825.294876
temp:0.0005585812037424937|epoch:0|change:4315933393514.8423|value:31538672825.294876
temp:0.0005585812037424937|epoch:5|change:1404391381931.8

temp:0.0004273047751308624|epoch:5|change:12566906193449.922|value:31538672825.294876
temp:0.00042548665035634537|epoch:0|change:7832746279752.926|value:31538672825.294876
temp:0.00042548665035634537|epoch:5|change:17717187005696.586|value:31538672825.294876
temp:0.00042368393178688723|epoch:0|change:16474397935620.943|value:31538672825.294876
temp:0.00042368393178688723|epoch:5|change:20717954186557.566|value:31538672825.294876
temp:0.00042189642442780323|epoch:0|change:29302850597957.934|value:31538672825.294876
temp:0.00042189642442780323|epoch:5|change:395839870597.53864|value:31538672825.294876
temp:0.00042012393656128583|epoch:0|change:43016129859468.734|value:31538672825.294876
temp:0.00042012393656128583|epoch:5|change:11843659498177.2|value:31538672825.294876
temp:0.0004183662796778582|epoch:0|change:7256961053897.859|value:31538672825.294876
temp:0.0004183662796778582|epoch:5|change:2040813860263.766|value:31538672825.294876
temp:0.00041662326840954093|epoch:0|change:55976384

temp:0.000333305557870178|epoch:5|change:27454820019621.887|value:31538672825.294876
temp:0.0003321983223984724|epoch:0|change:26790708123069.355|value:31538672825.294876
temp:0.0003321983223984724|epoch:5|change:270415714921.78766|value:31538672825.294876
temp:0.0003310984190050497|epoch:0|change:100266601193238.88|value:31538672825.294876
temp:0.0003310984190050497|epoch:5|change:105256383771.34647|value:31538672825.294876
temp:0.00033000577510106473|epoch:0|change:3111209627913.4756|value:31538672825.294876
temp:0.00033000577510106473|epoch:5|change:11890578967679.02|value:31538672825.294876
temp:0.00032892031905270994|epoch:0|change:11975521107658.281|value:31538672825.294876
temp:0.00032892031905270994|epoch:5|change:12758818295394.742|value:31538672825.294876
temp:0.00032784198016556063|epoch:0|change:33189367731239.113|value:31538672825.294876
temp:0.00032784198016556063|epoch:5|change:17599147519077.996|value:31538672825.294876
temp:0.00032677068866922683|epoch:0|change:2766247

temp:0.0002873356799080528|epoch:5|change:11193505044679.986|value:31538672825.294876
temp:0.000286512427476542|epoch:0|change:9217327108322.264|value:31538672825.294876
temp:0.000286512427476542|epoch:5|change:720483047747.0438|value:31538672825.294876
temp:0.0002856938790086424|epoch:0|change:26925960933955.31|value:31538672825.294876
temp:0.0002856938790086424|epoch:5|change:105623831470347.72|value:31538672825.294876
temp:0.00028487999430240027|epoch:0|change:22595802531292.535|value:31538672825.294876
temp:0.00028487999430240027|epoch:5|change:4216426469428.0186|value:31538672825.294876
temp:0.0002840707336126697|epoch:0|change:24134293480901.44|value:31538672825.294876
temp:0.0002840707336126697|epoch:5|change:40512757066651.125|value:31538672825.294876
temp:0.00028326605764464287|epoch:0|change:2817105627469.8784|value:31538672825.294876
temp:0.00028326605764464287|epoch:5|change:17831887679248.746|value:31538672825.294876
temp:0.00028246592754748974|epoch:0|change:1156150548853

temp:0.00025124049996859486|epoch:5|change:16468072717644.814|value:31538672825.294876
temp:0.0002506108639809535|epoch:0|change:10874913522388.816|value:31538672825.294876
temp:0.0002506108639809535|epoch:5|change:9741610582460.166|value:31538672825.294876
temp:0.0002499843759765014|epoch:0|change:7722355076964.763|value:31538672825.294876
temp:0.0002499843759765014|epoch:5|change:15390285129189.873|value:31538672825.294876
temp:0.00024936101240571033|epoch:0|change:23716065900219.3|value:31538672825.294876
temp:0.00024936101240571033|epoch:5|change:2337344642264.2866|value:31538672825.294876
temp:0.0002487407499533611|epoch:0|change:3644917057762.0938|value:31538672825.294876
temp:0.0002487407499533611|epoch:5|change:5679791601863.002|value:31538672825.294876
temp:0.0002481235655356367|epoch:0|change:31927295529182.547|value:31538672825.294876
temp:0.0002481235655356367|epoch:5|change:47155673918735.96|value:31538672825.294876
temp:0.0002475094362972588|epoch:0|change:28173168744560.

temp:0.0002217171997117676|epoch:5|change:3618071624787.764|value:31538672825.294876
temp:0.00022122670206293897|epoch:0|change:9965113692064.008|value:31538672825.294876
temp:0.00022122670206293897|epoch:5|change:9159841863572.117|value:31538672825.294876
temp:0.00022073836984713862|epoch:0|change:8932340819666.424|value:31538672825.294876
temp:0.00022073836984713862|epoch:5|change:3910425217279.174|value:31538672825.294876
temp:0.00022025218875612574|epoch:0|change:27536755659197.86|value:31538672825.294876
temp:0.00022025218875612574|epoch:5|change:24913960719458.098|value:31538672825.294876
temp:0.00021976814460743913|epoch:0|change:6998488816380.535|value:31538672825.294876
temp:0.00021976814460743913|epoch:5|change:30597152747203.176|value:31538672825.294876
temp:0.00021928622334301844|epoch:0|change:9545920646239.982|value:31538672825.294876
temp:0.00021928622334301844|epoch:5|change:21303966163606.254|value:31538672825.294876
temp:0.00021880641102784307|epoch:0|change:630604911

In [27]:
solutions = {"uid_1":[], "uid_2":[],
             "mat_1":[], "mat_2":[],
             "tA":[],    "tB":[],
             "new":[],   "strainB":[],
             "new_uid":[],       }

In [41]:
structure_path = "/Users/voicutu/Documents/GitHub/vdw_evolve/sandbox/XYZcolection"
saving_spot = 5
to_be_generated = 10
while to_be_generated!=0:
    
    data1= raw_df_1.iloc[random.randinit(0,len(raw_df_1["uid"]))
    data2= raw_df_1.iloc[random.randinit(0,len(raw_df_2["uid"]))
    uid_1 = data1["uid"]
    uid_2 = data2["uid"]
    
    new_uid =uid_1+"_"+uid_2
    
    if new_uid not in solutions["new_uid"]:
        to_be_generated = to_be_generated-1
        
        solutions["new_uid"].append(new_uid)
        solutions["uid_1"].append(uid_1)
        solutions["uid_2"].append(uid_2)
        
        # calculate the super cel 
        
        # xyz path 
        path1=structure_path+"/"+uid_1+".json"
        path2=structure_path+"/"+uid_2+".json"
        
        # lattices
        latice1 = get_lattice_from_structure(path1)
        latice2 = get_lattice_from_structure(path2)


        cel1 = [[latice1[0][0], latice1[1][0]],
                [latice1[0][1], latice1[1][1]]]

        cel2 = [[latice2[0][0], latice2[1][0]],
                [latice2[0][1], latice2[1][1]]]
        
        tA, tB, strain = build_annealing(cel1,cel2, 5, model_par)
        new = np.dot(tA, cel1)
        
        solutions["tA"].append(tA)
        solutions["tB"].append(tB)
        solutions["new"].append(new)
        solutions["strainB"].append(strain)
    
    # save
    if k%saving_spot==0:
        data = pd.DataFrame.from_dict(solutions)
        data.to_csv("SuperCell_"+str(len(solutions["new"])))

print("Done")

SyntaxError: invalid syntax (4129823162.py, line 9)